In [228]:
import openrouteservice as ors
import folium 
import operator
from functools import reduce
from pprint import pprint

In [229]:
def add_marker(cords, message, color):
    folium.Marker(
        location=list(reversed(cords)),
        popup=message,
        icon=folium.Icon(color=color),
        ).add_to(m)
    return

In [230]:
def add_line(path, color="blue"):
    folium.PolyLine(locations=[list(reversed(coord)) for coord in path['features'][0]['geometry']['coordinates']], color=color).add_to(m)
    return

In [231]:
min_sum = 0
src_minval=99
des_minval=99
# ors format - [long, lat]
# folium - [lat, long]
# api key setup 
client = ors.Client(key='5b3ce3597851110001cf62480f73447f67a2444fa6cc065b11091e9f')
# default location to load map
m = folium.Map(location=list(reversed([72.8429,19.0194])), tiles="cartodbpositron", zoom_start=13)

# user coords
user_cords = [[72.830098,18.964234], [72.836745, 18.992558]]

user_route = client.directions(coordinates=user_cords,
                          profile='foot-walking',
                          format='geojson')

user_route_steps = user_route['features'][0]['geometry']['coordinates']
ucords = np.array(user_cords)
# add user line and markers
add_line(path=user_route)
add_marker(cords=user_cords[0], message=f'User-start {user_cords[0]}', color="orange")
add_marker(cords=user_cords[1], message=f'User-end {user_cords[1]}', color="purple")

# route coords
coords = [
    [[72.839412, 18.944179], [72.8362869, 19.0240881]],
    [[72.879096, 19.038091], [72.844032, 19.030788]],
    [[72.834946, 18.981618], [72.809643, 18.980450]]
]

for i in range(len(coords)):
    print(coords[i])
    route = client.directions(coordinates=coords[i],
                              profile='foot-walking',
                              format='geojson')
    
    route_steps = route['features'][0]['geometry']['coordinates']

    add_line(path=route)
    add_marker(cords=coords[i][0], message=f'Route-start {coords[i][0]}', color="green")
    add_marker(cords=coords[i][1], message=f'Route-end {coords[i][1]}', color="red")

    src_route_start = client.directions(coordinates=[user_cords[0], route_steps[0]],
                              profile='foot-walking',
                              format='geojson')
    des_route_end = client.directions(coordinates=[user_cords[1], route_steps[-1]],
                              profile='foot-walking',
                              format='geojson')
    print(src_route_start["features"][0]["properties"]["segments"][0]["distance"], des_route_end["features"][0]["properties"]["segments"][0]["distance"])

    steps = np.array(route_steps)
    src_diff = np.sum(np.abs(steps-ucords[0]),axis=1)
    des_diff = np.sum(np.abs(steps-ucords[1]),axis=1)

    add_line(path=route)
    add_marker(cords=coords[i][0], message=f'Route-start {coords[i][0]}', color="green")
    add_marker(cords=coords[i][1], message=f'Route-end {coords[i][1]}', color="red")

    min_src = np.argmin(src_diff)
    min_des = np.argmin(des_diff)

    if src_minval>src_diff[min_src]:
        src_minval = src_diff[min_src]
        min_src_cord = steps[min_src]
    if des_minval>des_diff[min_des]:
        des_minval = des_diff[min_des]
        min_des_cord = steps[min_des]

    print(src_minval, des_minval, min_src_cord, min_des_cord)

add_marker(cords=min_src_cord, message=f"Minsrc- {min_src_cord}", color="black")
add_marker(cords=min_des_cord, message=f"Mindes- {min_des_cord}", color="black")
# add_line(path=client.directions([list(min_src_cord), list(min_des_cord)], 'foot-walking', 'geojson'), color="black")

m

[[72.839412, 18.944179], [72.8362869, 19.0240881]]
3401.5 4379.8
0.0026720000000146626 0.0028259999999953322 [72.828026 18.963634] [72.836731 18.99537 ]
[[72.879096, 19.038091], [72.844032, 19.030788]]
14331.0 5720.0
0.0026720000000146626 0.0028259999999953322 [72.828026 18.963634] [72.836731 18.99537 ]
[[72.834946, 18.981618], [72.809643, 18.98045]]
2809.7 4760.5
0.0026720000000146626 0.0028259999999953322 [72.828026 18.963634] [72.836731 18.99537 ]


In [209]:
src_diff

array([0.022143, 0.022055, 0.021158, 0.020582, 0.019084, 0.018954,
       0.018718, 0.018572, 0.018508, 0.018546, 0.018397, 0.018246,
       0.018287, 0.018262, 0.017909, 0.017719, 0.017645, 0.017409,
       0.017264, 0.016885, 0.016681, 0.016332, 0.015875, 0.016817,
       0.017075, 0.017132, 0.017037, 0.017163, 0.018556, 0.018669,
       0.019264, 0.019875, 0.020209, 0.020347, 0.022098, 0.022619,
       0.022832, 0.023013, 0.023135, 0.023141, 0.023314, 0.023471,
       0.024317, 0.024403, 0.024431, 0.024351, 0.023862, 0.02396 ,
       0.024078, 0.024385, 0.024529, 0.024642, 0.02479 , 0.025339,
       0.026152, 0.026395, 0.028231, 0.028943, 0.028835, 0.029251,
       0.031712, 0.03225 , 0.032293, 0.032419, 0.033221, 0.032653,
       0.032638, 0.033075, 0.033059, 0.033015, 0.033367, 0.033966,
       0.034413, 0.034633, 0.03472 , 0.03478 , 0.03502 , 0.03509 ,
       0.035165, 0.036242, 0.036387, 0.036586, 0.036671])

In [144]:
route["features"][0]["properties"]["segments"][0]["distance"]

1733.8

In [159]:
route_steps[-1]

[72.8363, 19.02404]

In [135]:
a = np.array([72.828141,18.966171])
user_cords[0]

[72.830098, 18.964234]

In [136]:
a - user_cords[0]

array([-0.001957,  0.001937])

In [20]:
import folium


# Sample data
routes = [
    {
        "route_id": 1,
        "start_location": {
            "latitude": 72.834614,
            "longitude": 19.1073961
        },
        "end_location": {
            "latitude": 72.8543861,
            "longitude": 19.275928
        }
    },
    {
        "route_id": 2,
        "start_location": {
            "latitude": 72.8429,
            "longitude": 19.0194
        },
        "end_location": {
            "latitude": 75.3433,
            "longitude": 19.8762
        }
    },
    {
        "route_id": 3,
        "start_location": {
            "latitude": 72.8572,
            "longitude": 19.2295
        },
        "end_location": {
            "latitude": 73.7939,
            "longitude": 20.0059
        }
    }
]


# Create a Folium map centered at the first start_location
# m = folium.Map(location=[routes[0]["start_location"]["latitude"], routes[0]["start_location"]["longitude"]], zoom_start=7)

# Plot route points and polylines
for route in routes:
    folium.Marker(
        location=[route["start_location"]["latitude"], route["start_location"]["longitude"]],
        popup=f"Route {route['route_id']} - Start",
        icon=folium.Icon(color="green"),
    ).add_to(m)

    folium.Marker(
        location=[route["end_location"]["latitude"], route["end_location"]["longitude"]],
        popup=f"Route {route['route_id']} - End",
        icon=folium.Icon(color="red"),
    ).add_to(m)
    
    coords = [
            [route["start_location"]["latitude"], route["start_location"]["longitude"]],
            [route["end_location"]["latitude"], route["end_location"]["longitude"]]
            ]
#     print(coords)
    m = folium.Map(location=list(reversed([-77.0362619, 38.897475])), tiles="cartodbpositron", zoom_start=13)
    path = client.directions(coordinates=coords,
                          profile='driving-car',
                          format='geojson')
#     print(path)
#     break
    
    folium.PolyLine(locations=[list(reversed(coord)) for coord in path['features'][0]['geometry']['coordinates']], color="blue").add_to(m)

m